<a href="https://colab.research.google.com/github/tekliyetamiru/Natural-Language-Processing-NLP-with-TensorFlow/blob/main/Natural_Language_Processing_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction of NLP Fundamentals in TensorFlow
NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problem is sequence to sequence problems(seq2seq)

# Check for GPU

In [ ]:
!nvidia-smi -L

## Get helper functions

In [1]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves, compare_historys

--2025-11-06 09:31:37--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2025-11-06 09:31:37 (112 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text Dataset

The dataset we're going to be using is kaggle's introduction to NLP dataset ( text samples of Tweets labelled as disaster or not disaster).

See the orginal here: https://www.kaggle.com/c/nlp-getting-started

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-11-06 09:32:03--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.207, 142.251.111.207, 142.251.163.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2025-11-06 09:32:03 (77.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

To visualize out text samples,we first have to read them in, one way to do so would be to use python: https://realpython.com/read-write-files-python/

But I prefer to get visual straight way.
So another way to do this is to use pandas...

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [5]:
# shuffle training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [ ]:
# what does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
# how many examples of each class?
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [ ]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [6]:
# Let's visualize some random training examples
import random
random_index=random.randint(0,len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","(real diaster)" if target>0 else "(not real diaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target:1 (real diaster)
Text:
After shooting event at the theater @Starbucks is giving FREE COFFEE to all first responders police &amp; firefighters. AntiochHickoryHollow#TN

---

Target:1 (real diaster)
Text:
News@ Refugio oil spill may have been costlier bigger than projected http://t.co/jhpdSSVhvE

---

Target:1 (real diaster)
Text:
But if you build an army of 100 dogs and their leader is a lion all dogs will fight like a lion.

---

Target:1 (real diaster)
Text:
Australia's Ashes disaster - how the collapse unfolded at Trent Bridge... http://t.co/Dq3ddGvgBF
 #cricket

---

Target:1 (real diaster)
Text:
http://t.co/wspuXOrEWb  Cindy Noonan@CindyNoonan-Heartbreak in #Baltimore #Rioting #YAHIstorical #UndergroundRailraod

---



### Split data into training and validation sets

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
# Use train_test_split to split training data into training and validation sets
train_sentences,val_sentences,train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                          train_df_shuffled["target"].to_numpy(),
                                                                          test_size=0.1, #use 10% of training data for valid
                                                                          random_state=42)


In [9]:
# Check the lengths
len(train_sentences), len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [ ]:
# Check the first 10 samples
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll to do before you can build a model is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenziation - direct mapping of token (a token could be a word or a character) to number
* Embedding - create a matrix of teature vector for each token(this size of the features vector can be defined and this embedding can be learned)


### Text Vectorization(tokenization)

In [ ]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [10]:
from os.path import split
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization


# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens=10000, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # Create groups of n-words?
                                    output_mode="int", # how to map token to numbers
                                    output_sequence_length=15, # how long do you want your sequences to be?
                                    pad_to_max_tokens=True)



In [ ]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [11]:
# Find the average number of token (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [12]:
for i in train_sentences[:5]:
  print(i.split(),"\n")


['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet'] 

['Imagine', 'getting', 'flattened', 'by', 'Kurt', 'Zouma'] 

['@Gurmeetramrahim', '#MSGDoing111WelfareWorks', 'Green', 'S', 'welfare', 'force', 'ke', 'appx', '65000', 'members', 'har', 'time', 'disaster', 'victim', 'ki', 'help', 'ke', 'liye', 'tyar', 'hai....'] 

['@shakjn', '@C7', '@Magnums', 'im', 'shaking', 'in', 'fear', "he's", 'gonna', 'hack', 'the', 'planet'] 

['Somehow', 'find', 'you', 'and', 'I', 'collide', 'http://t.co/Ee8RpOahPk'] 



In [13]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vacabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a tweet does a model see?)

text_vectorizer=TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)



In [14]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [15]:
# Create a sample sentence and tokenize it
sample_sentence= "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [16]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence=random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \\n\nVectorized Version:")
text_vectorizer([random_sentence])

Original text:
 New post: 'China's Stock Market Crash: Are There Gems In The Rubble?' http://t.co/6CaDRhIOxp \n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  50,  371, 1034,  413,  457,   85,   22,   74,  948,    4,    2,
         643,    1,    0,    0]])>

In [17]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words

print(f"Number of words in vocab:{len(words_in_vocab)}")
print(f"5 most common words:{top_5_words}")
print(f"5 least common words:{bottom_5_words}")

Number of words in vocab:10000
5 most common words:['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
5 least common words:[np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


## Creating an Embedding using an Embedding Layer

To make out embedding, we're going to use TensorFlow's embedding layer: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding


The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer


In [18]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # output shape
                             input_length=max_length # how long is each input
                             )
embedding

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


<Embedding name=embedding, built=False>

In [19]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 SANTA CRUZ ÛÓ Head of the St Elizabeth Police Superintendent Lanford Salmon has r ... - http://t.co/vplR5Hka2u http://t.co/SxHW2TNNLf      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.02254698, -0.01529747, -0.04026292, ..., -0.04879695,
         -0.01546868, -0.02066312],
        [ 0.0469822 , -0.0485322 , -0.03332677, ..., -0.01815221,
          0.01113328,  0.02973194],
        [ 0.02528507, -0.02507255,  0.03751949, ..., -0.03365185,
         -0.01173147,  0.04349352],
        ...,
        [ 0.04993265,  0.04972691, -0.04445237, ..., -0.04683855,
          0.00716748, -0.04231855],
        [-0.02730291, -0.01704217, -0.0363038 , ..., -0.02430866,
          0.00850027, -0.0203838 ],
        [-0.04552433,  0.00688466, -0.01036749, ..., -0.01279746,
          0.03995231,  0.01932973]]], dtype=float32)>

In [20]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]


(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 2.25469805e-02, -1.52974725e-02, -4.02629152e-02, -2.34591495e-02,
         2.08065398e-02, -3.73053774e-02, -4.41240557e-02,  2.71911509e-02,
         3.67575325e-02,  4.70730774e-02, -2.37334892e-03,  3.98466103e-02,
         4.55640815e-02, -2.86535267e-02, -2.66825035e-03, -1.40486583e-02,
         1.22405216e-03, -1.17247812e-02, -4.81984615e-02,  3.46757099e-03,
         1.11648068e-02, -4.74210978e-02, -1.98038574e-02,  4.44156788e-02,
         1.34517215e-02, -4.92140055e-02, -1.24256723e-02,  1.24413855e-02,
        -3.50968912e-03,  3.91652249e-02, -4.62975018e-02,  2.07314752e-02,
        -1.61721110e-02,  1.15362033e-02,  6.77246973e-03,  4.13731374e-02,
         3.13720964e-02, -1.53576732e-02, -8.38897377e-03,  4.75639962e-02,
         2.38332264e-02, -4.23616283e-02,  1.89413317e-02,  1.87612288e-02,
        -3.35686356e-02,  4.48053740e-02,  3.89728881e-02, -3.19209248e-02,
        -4.13882956e-02,  7.79756159e-0

## Modelling a text dataset (running a series of experiments)

Now we've a got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We'll start with a baseline and move on from there.

* Model 0: Naive Bayes (baseline), this is from sklearn ML
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model (RNN)
* Model 3: GRU model (RNN)
* Model 4: Bidirectional-LSTM mode (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to apporach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate our model



### Model 0: Getting a baseline

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> 🔑 **Note:** Is's common practice to use non-DL algorithms as a baseline because of their speed and then later using DL to see if you can improve upon them

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([("tfidf",TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf",MultinomialNB()) # model the text
                    ])

# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [22]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}% ")

Our baseline model achieves an accuracy of: 79.27% 


In [23]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function  for our model experiments

We could evaluate all of our model's predictions with different metrics every time, however, this will be cumbersome and could easily be fixed with a function.

Let's create one to compare our model's predictions with the truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score


In [24]:
# Function to evaluate :accuracy,precision,recall,f1-score
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  """
  Calculates mode accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100
  # Calculate model precision, recall and f1-score using "weighted" average
  model_predicison, model_recall, model_f1, _ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
  model_results = {"accuracy" : model_accuracy,
                   "precision" : model_predicison,
                   "recall": model_recall,
                   "f1" : model_f1}

  return model_results

In [25]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: A simple dense model

In [26]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype=tf.string) # input are 1-dimensional strings
x=text_vectorizer(inputs) # turn the input text into numbers
x=embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x)   # 👈 condense the feature vector for each token to one vector
outputs = layers.Dense(1,activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
inputs,outputs

(<KerasTensor shape=(None, 1), dtype=string, sparse=False, ragged=False, name=keras_tensor_4>,
 <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_8>)

In [27]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [28]:
# Fit the model
mode_1_history= model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20251106-093447
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6525 - loss: 0.6506 - val_accuracy: 0.7507 - val_loss: 0.5363
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8109 - loss: 0.4590 - val_accuracy: 0.7835 - val_loss: 0.4725
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8617 - loss: 0.3513 - val_accuracy: 0.7848 - val_loss: 0.4633
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8813 - loss: 0.2985 - val_accuracy: 0.7861 - val_loss: 0.4657
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9179 - loss: 0.2265 - val_accuracy: 0.7848 - val_loss: 0.4783


In [29]:
#Check the results
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7798 - loss: 0.5096


[0.4782865047454834, 0.7847769260406494]

In [30]:
# make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


(762, 1)

In [31]:
# look at a single prediction
model_1_pred_probs[0]

array([0.31399325], dtype=float32)

In [32]:
# look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.31399325],
       [0.7469082 ],
       [0.99757683],
       [0.12789848],
       [0.12047125],
       [0.94397134],
       [0.9186547 ],
       [0.9923089 ],
       [0.96633387],
       [0.28671074]], dtype=float32)

In [33]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [34]:
# Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7900209877855079,
 'recall': 0.7847769028871391,
 'f1': 0.781600538849599}

In [ ]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [35]:
# Looks like out baseline is out performing out first deep learning model...
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## visualizing learned embeddings

In [36]:
# Get the vocabulary from the text vectoriazation layer
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [41]:
# Model 1 Summary
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,840,389 (14.65 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,560,260 (9.77 MB)

In [49]:
# Get the weight matrix of embedding
# (these are the numericl representations of each token in our training data which have been learned for -5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape # same size as vocab size and embedding_dim (output_dim of our embedding layer)

(10000, 128)

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it.

To do so, Tensorflow has a handy tool called projector: http://projector.tensorflow.org/


And TensorFlow also has an incredible guide on word embedd

In [51]:
# Create embedding files (we got this from TensorFlow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv','w', encoding='utf-8')
out_m = io.open('metadata.tsv','w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue # skip 0 it's padding.
  vec=embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()